In [1]:
!pip install -q ultralytics optuna PyYAML tqdm
!pip install -U ultralytics
!pip install "numpy<2" --force-reinstall
!pip install --upgrade --force-reinstall scipy==1.11.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [2]:
import zipfile
import os
import shutil

zip_path = "/kaggle/input/bdd-in-yolo-fromat/yolo_data.zip"
extract_to = "/kaggle/working/yolo_data"

os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Unzipped raw data to:", extract_to)

splits = ['train', 'val', 'test']
for root, dirs, files in os.walk(extract_to):
    for f in files:
        if f.lower().endswith(('.jpg', '.jpeg', '.png', '.txt')):
            src_path = os.path.join(root, f)
            dest_dir = None
            for split in splits:
                if split in root.lower():
                    if f.lower().endswith(('.jpg', '.jpeg', '.png')):
                        dest_dir = os.path.join(extract_to, f"images/{split}")
                    else:  # .txt labels
                        dest_dir = os.path.join(extract_to, f"labels/{split}")
                    break
            if dest_dir:
                os.makedirs(dest_dir, exist_ok=True)
                shutil.move(src_path, os.path.join(dest_dir, f))

for root, dirs, files in os.walk(extract_to, topdown=False):
    for d in dirs:
        folder_path = os.path.join(root, d)
        if not os.listdir(folder_path):
            os.rmdir(folder_path)

Unzipped raw data to: /kaggle/working/yolo_data


In [3]:
!wget -O /kaggle/working/train.py "https://raw.githubusercontent.com/moaz-mamdouh207/Real-Time-Object-Detection-for-Autonomous-Vehicles/refs/heads/main/model/train.py"

from train import *

--2025-11-21 01:37:35--  https://raw.githubusercontent.com/moaz-mamdouh207/Real-Time-Object-Detection-for-Autonomous-Vehicles/refs/heads/main/model/train.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3789 (3.7K) [text/plain]
Saving to: ‘/kaggle/working/train.py’

/kaggle/working/tra 100%[===================>]   3.70K  --.-KB/s    in 0s      

2025-11-21 01:37:35 (29.3 MB/s) - ‘/kaggle/working/train.py’ saved [3789/3789]

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
model = 'yolov8s.pt'
best_params = optimize(model,5,10)

[I 2025-11-21 01:37:40,979] A new study created in memory with name: no-name-67237984-54c5-487a-81ea-5bca35b2f530
/kaggle/working/train.py:55: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr0 = trial.suggest_loguniform('lr0', 1e-4, 1e-1)


Ultralytics 8.3.229 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/yolo_data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.013523486462426699, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective

[W 2025-11-21 01:38:07,295] Trial 0 failed with parameters: {'lr0': 0.013523486462426699, 'batch_size': 32, 'img_size': 768} because of the following error: OutOfMemoryError('CUDA out of memory. Tried to allocate 144.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 32.12 MiB is free. Process 3324 has 14.71 GiB memory in use. Of the allocated memory 14.50 GiB is allocated by PyTorch, and 92.17 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)').
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/kaggle/working/train.py", line 95, in <lambda>
    study.optimize(lambda trial: objective(tria

OutOfMemoryError: CUDA out of memory. Tried to allocate 144.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 32.12 MiB is free. Process 3324 has 14.71 GiB memory in use. Of the allocated memory 14.50 GiB is allocated by PyTorch, and 92.17 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
train_and_export_onnx(model, best_params, epochs=50)